# Synthetic Data Generation: All Models
This notebook loads train data and runs each generator, saving synthetic datasets and logging stats.

In [4]:
!pip install sdv

  Using cached graphviz-0.20.3-py3-none-any.whl.metadata (12 kB)
  Using cached copulas-0.12.2-py3-none-any.whl.metadata (9.4 kB)
  Using cached ctgan-0.11.0-py3-none-any.whl.metadata (10 kB)
  Using cached deepecho-0.7.0-py3-none-any.whl.metadata (10 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached filelock-3.18.0-py3-none-any.whl.metadata (2.9 kB)
  Using cached sympy-1.14.0-py3-none-any.whl.metadata (12 kB)
  Using cached nvidia_cuda_nvrtc_cu12-12.6.77-py3-none-manylinux2014_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_runtime_cu12-12.6.77-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)
  Using cached nvidia_cuda_cupti_cu12-12.6.80-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cudnn_cu12-9.5.1.17-py3-none-manylinux_2_28_x86_64.whl.metadata (1.6 kB)
  Using cached nvidia_cublas_cu12-12.6.4.1-py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl.metadata (1.5 kB)

In [ ]:
import sys
import os
sys.path.append(os.path.abspath("..")) 
import pandas as pd
from pathlib import Path
from src.generators.tvae_generator import TVAESynthesizerWrapper
from src.generators.ctgan_generator import CTGANSynthesizerWrapper
from src.generators.ctabgan_generator import CTABGANSynthesizerWrapper
from src.generators.great_generator import GREATSynthesizerWrapper
from src.generators.rtf_generator import RTFGeneratorWrapper


Exception ignored in: <bound method IPythonKernel._clean_thread_parent_frames of <ipykernel.ipkernel.IPythonKernel object at 0x7f277ff05f50>>
Traceback (most recent call last):
  File "/home/jovyan/paper-realtab/lib/python3.11/site-packages/ipykernel/ipkernel.py", line 775, in _clean_thread_parent_frames
    def _clean_thread_parent_frames(

KeyboardInterrupt: 


In [13]:
# Load preprocessed training data
df = pd.read_csv("../data/processed/cirrhosis_train.csv")
dataset_name = "cirrhosis"

In [14]:
# Output log file
log_path = Path("../results/logs/synthetic_generation_log.csv")
log_path.parent.mkdir(parents=True, exist_ok=True)

In [16]:
# Define list of generators
generators = {
    "tvae": TVAESynthesizerWrapper(output_dir="../data/synthetic/tvae"),
    "ctgan": CTGANSynthesizerWrapper(output_dir="../data/synthetic/ctgan"),
    "ctabgan": lambda: CTABGANSynthesizerWrapper(output_dir="../data/synthetic/ctabgan", num_experiments=1),
    "great": GREATSynthesizerWrapper(output_dir="../data/synthetic/great"),
    "rtf": RTFGeneratorWrapper(output_dir="../data/synthetic/rtf")
    
}

In [17]:
# Defining CTABGAN config separately
ctabgan_configs = {
    "diabetes": {
        "raw_csv_path": "../data/processed/diabetes_train.csv",
        "categorical_columns": ['gender', 'hypertension', 'heart_disease', 'smoking_history', 'diabetes'],
        "log_columns": [],
        "mixed_columns": {},
        "general_columns": ['bmi', 'HbA1c_level'],
        "non_categorical_columns": [],
        "integer_columns": ['age', 'blood_glucose_level'],
        "problem_type": {"Classification": 'diabetes'}
    },
    "stroke": {
        "raw_csv_path": "../data/processed/stroke_train.csv",
        "categorical_columns": ['gender', 'hypertension', 'heart_disease', 'ever_married', 'work_type', 'Residence_type', 'smoking_status', 'stroke'],
        "log_columns": [],
        "mixed_columns": {},
        "general_columns": ['bmi'],
        "non_categorical_columns": [],
        "integer_columns": ['age', 'avg_glucose_level'],
        "problem_type": {"Classification": 'stroke'}
    },
    "cirrhosis": {
        "raw_csv_path": "../data/processed/cirrhosis_train.csv",
        "categorical_columns": ['Sex', 'Ascites', 'Hepatomegaly', 'Spiders', 'Edema', 'Drug', 'Status', 'Stage'],
        "log_columns": [],
        "mixed_columns": {},
        "general_columns": ['Cholesterol', 'Albumin', 'Copper', 'Alk_Phos', 'SGOT', 'Tryglicerides', 'Platelets', 'Prothrombin'],
        "non_categorical_columns": [],
        "integer_columns": ['N_Days', 'Age'],
        "problem_type": {"Classification": 'Status'}
    }
}


In [18]:
# Initialize logs
log_rows = []

In [19]:
# Run each generator
for name, generator in generators.items():
    print(f" Generating with {name.upper()}")
    if name == "ctabgan":
        config = ctabgan_configs.get(dataset_name.lower())
        if config is None:
            print(f"No CTABGAN config defined for dataset: {dataset_name}")
            continue
        generator_instance = generator()  # initialize via lambda
        synth_data, stats = generator_instance.fit_and_generate(df, dataset_name="cirrhosis", ctabgan_config=config)
    else:
        synth_data, stats = generator.fit_and_generate(df, dataset_name="cirrhosis")

    # Append to log
    log_rows.append({
        "model": name,
        "execution_time_sec": stats["execution_time_sec"],
        "peak_memory_mb": stats["peak_memory_mb"],
        "n_samples": len(synth_data)
    })

 Generating with TVAE
Detecting metadata from input dataframe...
 Synthetic data saved to: ../data/synthetic/tvae/cirrhosis_tvae.csv
 Execution Time: 7.86 seconds
 Peak Memory Usage: 1.18 MB
 Generating with CTGAN
Detecting metadata from input dataframe...
Initializing CTGAN synthesizer...
Starting model training...
Training complete.
Generating synthetic data...
Synthetic data saved to: ../data/synthetic/ctgan/cirrhosis_ctgan.csv
Execution time: 13.96 seconds
Peak memory usage: 1.21 MB
 Generating with CTABGAN
Initializing CTABGAN synthesizer...
Running experiment 1/1...


100%|██████████| 150/150 [00:32<00:00,  4.55it/s]


Finished training in 33.427955865859985  seconds.
Saved: ../data/synthetic/ctabgan/cirrhosis_ctabgan_0.csv
Training time: 33.43 seconds
Peak memory: 0.67 MB
 Generating with GREAT
Initializing GReaT synthesizer...
Starting training...


You're using a GPT2TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss


...

Step,Training Loss


  0%|          | 0/108 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 128 samples generated. Sampling efficiency is: 100.0000%
Critic round: 20,                     sensitivity_threshold: 0.02263888888888885,                         val_sensitivity: 0.012592592592592591,                             val_sensitivities: [0.002777777777777775, -0.008333333333333335, -0.01388888888888889, 0.02222222222222222, 0.013888888888888888, 0.047222222222222214, 0.024999999999999998, 0.02222222222222222, 0.02222222222222222, -0.008333333333333335, 0.0027777777777777775, 0.030555555555555558, -0.008333333333333335, 0.03333333333333333, 0.005555555555555553]
Generating synthetic data...


  0%|          | 0/220 [00:00<?, ?it/s]

Generated 0 invalid samples out of total 256 samples generated. Sampling efficiency is: 100.0000%
Saved synthetic data to: ../data/synthetic/rtf/cirrhosis_rtf.csv
Execution Time: 37.53 seconds
Peak Memory Usage: 167.52 MB


In [20]:
# Save log to CSV
log_df = pd.DataFrame(log_rows)
log_df.to_csv(log_path, index=False)
print(f" Log saved to {log_path}")

 Log saved to ../results/logs/synthetic_generation_log.csv


In [26]:
import os
os.getcwd()


'/home/jovyan/DataGen/Github/notebooks'

In [ ]:
import shutil

# Folder you want to download
folder_to_zip = '/home/jovyan/DataGen/Github'  # change this to your folder name
output_filename = 'github'  # this will create my_folder.zip

# Create the zip file
shutil.make_archive(output_filename, 'zip', folder_to_zip)
print(f"Zipped {folder_to_zip} as {output_filename}.zip")
